In [1]:
%load_ext autoreload  
%autoreload 2 
from subset_active_learning.subset_selection import select, preprocess, compare

/home/glai/anaconda3/envs/subset/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processed_ds = preprocess.preprocess_sst2("roberta-base")

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 776.68it/s]
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-e96fff4bcf800b80.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-c19c9b6a9c9f8182.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-8d2a9f8a95240b96.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-0c3dbb722d1f6005.arrow
Loading cache

In [3]:
small_batch_config = select.SubsetTrainingArguments(batch_size=8, learning_rate=1e-5, model_card="roberta-base")

In [4]:
subset_trainer = select.SubsetTrainer(params=small_batch_config, valid_ds=processed_ds["validation"], test_ds=processed_ds["test"])

In [5]:
seed = 0
n_downsample = 1000
sst2_downsample = processed_ds['train'].shuffle(seed=seed).select(range(0, n_downsample))
sst2_remaining = processed_ds['train'].shuffle(seed=seed).select(range(n_downsample, len(processed_ds['train'])))

Loading cached shuffled indices for dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-ffe29b54c11c2610.arrow


In [7]:
import wandb
wandb_run = wandb.init(
            project="subset-search-gpu-opt", entity="johnny-gary", tags=["7000_ds", small_batch_config.model_card]
        )
subset_trainer.train_one_step(subset=sst2_remaining, save_path="/home/glai/dev/subset-active-learning/local_bucket/7000_model.pt")

loss,▅▅▅▅▅▅▅▅▄▃▂▂▆▁▃▃▄▂▆▃▄▅▆▁▃▂█▃▃▁▁▁▁▁▇▆▁▇▁█
sst2_final_valid:accuracy,▁
sst:val_acc,▁█▆▆
loss,1.31893
sst2_final_valid:accuracy,0.86649
sst:val_acc,0.86649


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifi